### 문제 2-3 : 학생 정보 구조화 시스템
- 출력 파서: PydanticOutputParser
- 문제
학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 구조화된 형태로 추출하는 시스템을 만드세요.
- 요구사항:
    - PydanticOutputParser와 BaseModel 사용
    - 각 필드에 적절한 타입과 설명 추가
    - 자유 형식의 텍스트에서 정보 추출
- 예시 입력:
 "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
- 예시 출력 구조:
{
    "name": "김민수",
    "age": 22,
    "major": "컴퓨터공학",
    "hobbies": ["게임하기", "영화보기", "코딩"],
    "goal": "훌륭한 개발자가 되는 것"
}


In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser


# 1) Pydantic 모델 정의 (필드 타입/설명)
class StudentInfo(BaseModel):
    name: Optional[str] = Field(None, description="학생의 이름 (예: 김민수)")
    age: Optional[int] = Field(None, description="나이(정수). 예: 22")
    major: Optional[str] = Field(None, description="전공명. 예: 컴퓨터공학")
    hobbies: List[str] = Field(default_factory=list, description="취미 리스트. 예: ['게임하기','영화보기']")
    goal: Optional[str] = Field(None, description="학생의 목표. 예: 훌륭한 개발자가 되는 것")


# 2) 파서 준비
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 3) 추출 프롬프트
prompt = PromptTemplate(
    template=(
        "다음 자유 형식 자기소개에서 학생 정보를 구조화하라.\n"
        "- 반드시 한국어 입력을 처리한다.\n"
        "- '이름, 나이(정수), 전공, 취미 리스트, 목표'를 추출한다.\n"
        "- 취미는 쉼표/조사로 구분해 리스트로 만들고, 공백·중복 항목은 제거한다.\n"
        "- 나이는 숫자로만 기입(예: 22). 한국어 숫자(스물둘 등)가 있으면 정수로 변환한다.\n"
        "- 정보가 없으면 해당 필드는 비워 두거나 빈 리스트로 둔다.\n"
        "출력 형식은 아래 지시를 엄격히 따른다:\n\n"
        "{format_instructions}\n\n"
        "자기소개 원문:\n"
        "{text}\n"
    ),
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 4) LLM 설정 (원하는 엔드포인트 사용 가능)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

def extract_student_info(free_text: str) -> StudentInfo:
    _prompt = prompt.format(text=free_text)
    resp = llm.invoke(_prompt).content
    return parser.parse(resp)

# === 5) 테스트 ===
if __name__ == "__main__":
    sample = (
        "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. "
        "취미로는 게임하기, 영화보기, 코딩을 좋아합니다. "
        "앞으로 훌륭한 개발자가 되는 것이 목표입니다."
    )
    info = extract_student_info(sample)
    print(info.dict())